In [16]:
pip install pandas numpy matplotlib plotly nemosis nem_bidding_dashboard

In [17]:
import pandas as pd
import numpy as np
from datetime import timedelta, datetime
import matplotlib.pyplot as plt
from nemosis import static_table, dynamic_data_compiler
import plotly.express as px
import nem_bidding_dashboard

raw_data_cache = '/Volumes/T7/NEMO_data'

pd.set_option('display.max_columns', None)

# Methodology

Aim: Do a profit maximisation test for a single power station for a single auction

RD requires bids + dispatch load. It also requires finding the supply bids for all other firms.
MC requires bids + variable fuel costs


NEW:
1. As bids are on a per power station basis, adjust!

OLD:
1. Find MC estimates
2. Group units together by the firms that control them, including the firms's daily declared capacity
3. Create a total marginal cost function which represents the cost curve for all of a firm's generating units, stacked from lowest to highest cost. This creates a stepwise increasing function where: X-axis is cumulative MW across all units, Y-axis is marginal cost ($/MWh), each step represents a different generating unit. Width of step = unit's capacity. Height of step = unit's marginal cost
4. Taking only units that are verified to be "on-line" and operating during that hour
5. Subtracting the day-ahead scheduled quantity to center the function around 0
6. Including only natural gas and coal units that can respond quickly (excluding nuclear, wind, hydro)

#Step 1: MC estimates
Black coal: ~$41-101/MWh
Brown coal: ~$12-13/MWh
Wind/Solar: $0-1/MWh

Methodology:
1. Fuel cost range: 
2. Heat rate
3. Variable O&M (Operations and Maintenance)

Note: 
1. This includes a big assumption that MC is the same across every firm for each fuel type, which is not true!
2. For coal, it would be good to include a shutdown cost - I don't want to arbitrarily limit it like Hortaçsu as emperically coal firms are choosing to shut down

MC estimates:
X-axis: Cumulative quantity of electricity
Y-Axis: Price
MC:
Black coal: ~$41-101/MWh
Brown coal: ~$12-13/MWh
Natural Gas: $60-80/MWh

In [18]:
cost_estimates = {
    "Brown Coal": 12.5,       # Range $12–13 => Midpoint ~12.5
    "Black Coal": 71,         # Range $41–101 => Midpoint ~71
    "Natural Gas": 70,        # Range $60–80 => Midpoint ~70
    "Kerosene": 300,          # Range $200–400 => Midpoint ~300
    "Water": 10     # Range $0–20 => Midpoint ~10
#     "New Large-Scale Hydro": 95  # Range $40–150 => Midpoint ~95
}

In [19]:
# Required to join DUIDs to firm names 
generator_info_df = static_table(table_name='Generators and Scheduled Loads', 
                              raw_data_location=raw_data_cache,
                              update_static_file=False)
generator_info_df

# Find the marginal cost function and inelastic demand

In [20]:
# This is the table showing what electricity has been actually dispatched
dispatch_load_df = dynamic_data_compiler(start_time='2021/03/01 00:00:00',
                                   end_time='2021/04/10 00:00:00',
                                   table_name='DISPATCHLOAD',
                                   raw_data_location=raw_data_cache)
dispatch_load_df

In [21]:
# We will now merge the dispatch table with info from the Generator table
# Perform an outer join to ensure we keep all DUIDs and settlement dates
merged_dispatch_with_units_df = pd.merge(dispatch_load_df[['SETTLEMENTDATE', 'DUID']], generator_info_df, on="DUID", how="outer")

# Now merge with the full dispatch_load dataset to bring all the fields together
working_dispatch_df = pd.merge(merged_dispatch_with_units_df,dispatch_load_df, on=["DUID", "SETTLEMENTDATE"], how="outer")

In [22]:
working_dispatch_df.describe()

What are the steps to filter working_dispatch_df to scale up.

Preliminary:
Remove any rows where TOTALCLEARED == 0.0 as this means no electricity was dispatched, it was a balancing action.

time_date_company
Time is fixed and then a nested loop is needed to go through the different dates and companies.

In [23]:
print(working_dispatch_df.columns)

In [24]:
working_dispatch_df['TOTALCLEARED'].dtype

In [25]:
working_dispatch_df.shape

In [26]:
working_dispatch_df_filtered = working_dispatch_df[working_dispatch_df['TOTALCLEARED'].notna() & (working_dispatch_df['TOTALCLEARED'] != 0.0)]
working_dispatch_df_filtered.shape

In [27]:
# We now filter again just for the dispatch loads of Origin Energy electricity plants for the 6:00-6:05pm auction
dispatch_df_time = working_dispatch_df_filtered[
    working_dispatch_df_filtered["SETTLEMENTDATE"].astype(str).str.contains("18:05:00", na=False)
]

In [28]:
date = '2021-04-07'

In [29]:
# Filter for a single day
# Next step: create a for loop to go through each of the dates
dispatch_df_time_date = dispatch_df_time[dispatch_df_time['SETTLEMENTDATE'].astype(str).str.contains(date)]

## What is inelastic demand for this 5 minute auction on this day?

In [86]:
inelastic_demand = dispatch_df_time_date['TOTALCLEARED'].sum()
inelastic_demand

In [31]:
firm_i = "Origin Energy Electricity Limited"

In [32]:
# Filter for 'Origin Energy Electricity Limited' as it's the company with the largest number of power stations
# We join the dispatch_units and the dispatch_load tables on DUID
dispatch_df_time_date_company = dispatch_df_time_date[dispatch_df_time_date["Participant"] == firm_i]

dispatch_df_time_date_company

## Find the Marginal Cost Function

In [33]:
dispatch_df_time_date_company.loc[:, "AU$/MWh"] = (
    dispatch_df_time_date_company["Fuel Source - Descriptor"].map(cost_estimates)
)

dispatch_df_time_date_company

In [34]:
# Sorting by MC per MW/h as a firm would dispatch their lowest cost power stations first
dispatch_df_time_date_company = dispatch_df_time_date_company.sort_values("AU$/MWh")
dispatch_df_time_date_company

In [35]:
len(dispatch_df_time_date_company)

In [36]:
# What is firm i's capacity for this 5 minute auction on this day?
total_avail = dispatch_df_time_date_company['AVAILABILITY'].sum()
total_avail

In [37]:
# Compute the cumulative capacity using the 'MAXAVAIL_x' column
dispatch_df_time_date_company['CumulativeCapacity'] = dispatch_df_time_date_company['AVAILABILITY'].cumsum()

In [38]:
dispatch_df_time_date_company

In [39]:
# Plot the Marginal Cost curve for firm i

# 3. Plot the stepwise MC (price) function
plt.figure(figsize=(10, 6))

# Plot the step function (supply curve)
plt.step(
    dispatch_df_time_date_company['CumulativeCapacity'],
    dispatch_df_time_date_company['AU$/MWh'],
    where='post',
    label="MC Supply Curve"
)

# 4. Overlay cross markers for each data point
plt.plot(
    dispatch_df_time_date_company['CumulativeCapacity'],
    dispatch_df_time_date_company['AU$/MWh'],
    linestyle="None",   # no line, only markers
    marker="x",         # cross markers
    color="red",
    label="Data Points"
)

# 5. Label the axes
plt.xlabel("Quantity (MW)")
plt.ylabel("Price (AU$/MWh)")
plt.title(f"Marginal Cost Function for {firm_i} on {date}")

# Add a legend and grid for clarity
plt.legend()
plt.grid(True)

plt.show()

# Find the Supply Bid Function

Preliminary:
Remove any rows which are not energy actions as these are bids for the separate balancing market.

As with filtering the dispatch_df, we will follow the hierarchial structure of time_date_company
Time is fixed and then a nested loop is needed to go through the different dates and companies.

In [40]:
# We now need to find the supply bids to work out residual demand and the actual supply bid function
volume_bids = dynamic_data_compiler(start_time='2021/03/01 00:00:00',
                                   end_time='2021/04/10 00:00:00',
                                   table_name='BIDPEROFFER_D',
                                   raw_data_location=raw_data_cache)

In [41]:
# Join the dispatch_units and the dispatch_load tables on DUID

# Perform an outer join to ensure we keep all DUIDs and settlement dates
merged_bids_with_units_df = pd.merge(volume_bids, generator_info_df, on="DUID", how="outer")

In [42]:
filtered_merged_bids_with_units_df  = merged_bids_with_units_df[merged_bids_with_units_df['BIDTYPE'] == 'ENERGY']
filtered_merged_bids_with_units_df.shape

In [43]:
# Filter for only the 6-6:05pm auctions
# Here we look at the interval at 6pm because this is the start of the auction where the bids apply
bids_time_df = filtered_merged_bids_with_units_df[
    filtered_merged_bids_with_units_df["INTERVAL_DATETIME"].astype(str).str.contains("18:00:00", na=False)
]

In [44]:
# Filter for the single day we're looking at '2021-04-07'
# Eventually need to wrap this in a loop across all days

target_date = pd.to_datetime(date)
volume_bids_time_date_df = bids_time_df[bids_time_df['SETTLEMENTDATE'].dt.date == target_date.date()]

In [45]:
volume_bids_time_date_df

In [46]:
volume_bids_time_date_df.shape

In [47]:
# This is the number of DUIDS
# We need to first aggregate into the same market participants (firms)
# And then find the residual demand from each residual bid
unique_DUIDs = volume_bids_time_date_df["DUID"].unique()
len(unique_DUIDs)

In [48]:
unique_participants = volume_bids_time_date_df["Participant"].unique()
len(unique_participants)

In [49]:
# # Filter for 'Origin Energy Electricity Limited'
# filtered_bids_df = bids_with_duid[bids_with_duid["Participant"] == "Origin Energy Electricity Limited"]

What is the strategy for price bids?
Important to recall that price bids only change once a day
We need to match the price bands ups with volume bids based on the DUID
Can I just do this in one go per day?

Filtering:
Remove any bids that are not 'ENERGY'

In [50]:
# Get the price bids.
price_bids_df = dynamic_data_compiler(start_time='2021/03/01 00:00:00',
                                   end_time='2021/04/10 00:00:00',
                                   table_name='BIDDAYOFFER_D',
                                   raw_data_location=raw_data_cache)

In [51]:
price_bids_filtered_df = price_bids_df[price_bids_df['BIDTYPE'] == 'ENERGY']

In [52]:
price_bids_day_df = price_bids_filtered_df[price_bids_filtered_df['SETTLEMENTDATE'] == '2021-04-07']
price_bids_day_df

In [53]:
merged_volume_and_price_bids_df = pd.merge(price_bids_day_df, volume_bids_time_date_df, on=["DUID", "SETTLEMENTDATE"])
merged_volume_and_price_bids_df

In [54]:
# Remove where max avail is zero because these are placeholder bids
# Where the generator is not able to produce
merged_volume_and_price_bids_filtered_df = merged_volume_and_price_bids_df[merged_volume_and_price_bids_df["MAXAVAIL"] != 0]
merged_volume_and_price_bids_filtered_df

In [55]:
# What is the maximum Megawatts that are able to be produced in this auction
# if all generators generated at peak capacity
max_avail = merged_volume_and_price_bids_filtered_df['MAXAVAIL'].sum()
max_avail

There is a modification required because the total capacity that firms bid is often higher than the max capacity of the generator. This is because the max capacity takes priority and the auctioneer caps firms at their submitted max capacity. This means firms are able to bid whatever volumes in the knowledge they won't be called to produce more than their actual max capacity. 

We will have the columns:
MAXAVAIL: Hard limit
TOTAL_BANDAVAIL: Total volume bid
SUM_OF_BANDAVAIL: The capped volume bid so that what firms bid matches the hard limit 

In [56]:
# 1. Identify the band availability columns
bandavail_cols = [f"BANDAVAIL{i}" for i in range(1, 11)]  # BANDAVAIL1 to BANDAVAIL10

# 2. Create a new column summing them correctly with .loc to avoid SettingWithCopyWarning
merged_volume_and_price_bids_filtered_df = merged_volume_and_price_bids_filtered_df.copy()  # Ensure we are working on a copy
merged_volume_and_price_bids_filtered_df.loc[:, "TOTAL_BANDAVAIL"] = merged_volume_and_price_bids_filtered_df[bandavail_cols].sum(axis=1)

# 3. Remove rows where TOTAL_BANDAVAIL is 0
no_zeroes_combined_bids_df = merged_volume_and_price_bids_filtered_df[merged_volume_and_price_bids_filtered_df["TOTAL_BANDAVAIL"] != 0].copy()

In [57]:
"""
For each row in the group (same DUID & SETTLEMENTDATE):
  1) Sort the 10 band pairs by ascending price.
  2) Accumulate volumes, stopping at MAXAVAIL.
  3) Partially fill the band where we hit the limit.
  4) Set subsequent bands to zero.
  5) Write the modified band volumes back in the original wide order.
"""
    
def cap_bands_in_wide_mode(group):

    # We'll modify each row separately
    new_rows = []
    
    for _, row in group.iterrows():
        
        # Extract max_avail for this row
        max_avail = row["MAXAVAIL"]
        
        # Gather the 10 band pairs: (price, volume, band_index)
        # E.g., (PRICEBAND1, BANDAVAIL1, index=1), etc.
        band_info = []
        for i in range(1, 11):
            price_col = f"PRICEBAND{i}"
            vol_col   = f"BANDAVAIL{i}"
            
            price_val = row[price_col]
            vol_val   = row[vol_col]
            
            band_info.append((price_val, vol_val, i))
        
        # Sort by ascending price
        band_info.sort(key=lambda x: x[0] if not pd.isna(x[0]) else np.inf)
        
        # Accumulate volumes, capping at max_avail
        running_sum = 0.0
        capped_bands = []
        
        for (price, volume, idx) in band_info:
            if pd.isna(volume): 
                # If volume is NaN, treat it as 0
                volume = 0.0
            
            if running_sum >= max_avail:
                # Already at or beyond limit
                capped_bands.append((price, 0.0, idx))
            else:
                potential = running_sum + volume
                if potential <= max_avail:
                    # Can use full band
                    capped_bands.append((price, volume, idx))
                    running_sum += volume
                else:
                    # Partially use this band
                    remainder = max_avail - running_sum
                    capped_bands.append((price, remainder, idx))
                    running_sum += remainder
        
        # Now re-sort by the original band index so we can put them back into the row
        capped_bands.sort(key=lambda x: x[2])
        
        # Write them back into the row's band columns
        row_capped = row.copy()  # copy original
        for (price, cap_vol, idx) in capped_bands:
            vol_col = f"BANDAVAIL{idx}"
            row_capped[vol_col] = cap_vol
        
        new_rows.append(row_capped)
    
    # Return the modified rows as a DataFrame
    return pd.DataFrame(new_rows)

capped_combined_bids_df = (
    no_zeroes_combined_bids_df
    .groupby(["DUID", "SETTLEMENTDATE"], group_keys=False)
    .apply(cap_bands_in_wide_mode)
)

capped_combined_bids_df

In [58]:
# Define columns for melting
price_columns = [f"PRICEBAND{i}" for i in range(1, 11)]
volume_columns = [f"BANDAVAIL{i}" for i in range(1, 11)]

# Include the additional columns you want to keep in long format
id_vars_cols = [
    "DUID",
    "SETTLEMENTDATE",
    "Participant",
    "Station Name",
    "Fuel Source - Descriptor"
]

# Melt price bands into long format
melted_prices = (
    capped_combined_bids_df
    .melt(
        id_vars=id_vars_cols,             # <--- Add your additional columns here
        value_vars=price_columns,
        var_name="PRICE_BAND",
        value_name="PRICE"
    )
)

# Melt volume bands into long format
melted_volumes = (
    capped_combined_bids_df
    .melt(
        id_vars=id_vars_cols,             # <--- Same additional columns
        value_vars=volume_columns,
        var_name="VOLUME_BAND",
        value_name="VOLUME"
    )
)

# Extract the band number (1..10) from PRICE_BAND or VOLUME_BAND
melted_prices["BAND_NUMBER"] = melted_prices["PRICE_BAND"].str.extract(r"(\d+)").astype(int)
melted_volumes["BAND_NUMBER"] = melted_volumes["VOLUME_BAND"].str.extract(r"(\d+)").astype(int)

# Merge long-format prices and volumes on DUID, SETTLEMENTDATE, and BAND_NUMBER
bid_curve_df = pd.merge(
    melted_prices,
    melted_volumes,
    on=["DUID", "SETTLEMENTDATE", "Participant", "Station Name", "Fuel Source - Descriptor", "BAND_NUMBER"]
)

# View the merged DataFrame
bid_curve_df.head()

In [59]:
bid_curve_df.shape

In [60]:
bid_curve_df = bid_curve_df.sort_values(by=["DUID", "SETTLEMENTDATE", "PRICE"])
bid_curve_df["CUMULATIVE_VOLUME"] = bid_curve_df.groupby(["DUID", "SETTLEMENTDATE"])["VOLUME"].cumsum()
with pd.option_context('display.max_rows', None, 
                      'display.max_columns', None,
                      'display.width', None,
                      'display.max_colwidth', None):
    display(bid_curve_df)

In [61]:
# This aggregates volumes by (Participant, SETTLEMENTDATE, PRICE)
# meaning that we can construct a firm-wide aggregate supply function 
# which has multiple DUIDs

firm_aggregated_bids = (
    bid_curve_df
    .groupby(["Participant", "SETTLEMENTDATE", "PRICE"], as_index=False)["VOLUME"]
    .sum()
)

# 2) Sort so we can build a piecewise curve in ascending PRICE order
firm_aggregated_bids = firm_aggregated_bids.sort_values(["Participant", "SETTLEMENTDATE", "PRICE"])

firm_aggregated_bids["FIRM_CUMULATIVE_VOLUME"] = (
    aggregated_bids
    .groupby(["Participant", "SETTLEMENTDATE"])["VOLUME"]
    .cumsum()
)

In [62]:
firm_aggregated_bids["TOTAL_CUMULATIVE_VOLUME"] = firm_aggregated_bids.groupby("SETTLEMENTDATE")["VOLUME"].cumsum()

In [63]:
firm_aggregated_bids

## Finding the residual demand

In [64]:
firm_i_df = firm_aggregated_bids[firm_aggregated_bids["Participant"] == firm_i]
firm_i_df

In [65]:
# Plot firm i's supply bid function

plt.step(firm_i_df["FIRM_CUMULATIVE_VOLUME"], firm_i_df["PRICE"], where='post')
plt.xlabel("Cumulative Volume (MW)")
plt.ylabel("Price ($/MWh)")
plt.title(f"Supply Bid Function of {firm_i} on {date}")
plt.show()

In [ ]:
rival_firms_df = aggregated_bids[aggregated_bids["Participant"] != firm_i].copy()

In [66]:
rival_firms_df

In [67]:
rival_grouped_df = (
    rival_firms_df
    .groupby(["SETTLEMENTDATE", "PRICE"], as_index=False)["VOLUME"]
    .sum()
    .rename(columns={"VOLUME": "RIVAL_VOLUME"})  # rename for clarity
)

# Compute stepwise cumulative supply for each date
rival_grouped_df["CUMULATIVE_RIVAL_SUPPLY"] = (
    rival_grouped_df
    .groupby("SETTLEMENTDATE")["RIVAL_VOLUME"]
    .cumsum()
)

rival_grouped_df.head(20)

In [68]:
# We have already found the inelastic demand by summing the total cleared in dispatch
inelastic_demand

In [69]:
rival_grouped_df["RESIDUAL_DEMAND"] = (
    inelastic_demand 
    - rival_grouped_df["CUMULATIVE_RIVAL_SUPPLY"]
)

rival_grouped_df.head(50)

In [70]:
plt.figure(figsize=(8, 6))
plt.step(
    rival_grouped_df["RESIDUAL_DEMAND"],  # X-axis
    rival_grouped_df["PRICE"],           # Y-axis
    where='post'
)

plt.xlabel("Residual Demand (MW)")
plt.ylabel("Price ($/MWh)")
plt.title("Residual Demand Curve")
plt.grid(True)
plt.show()

## Plotting firm i's supply bid function, firm i's residual demand, and the marginal cost

In [71]:
plt.figure(figsize=(10, 6))

# 1. Plot the Residual Demand Curve (step plot)
plt.step(
    rival_grouped_df["RESIDUAL_DEMAND"],  # X-axis for residual demand
    rival_grouped_df["PRICE"],            # Y-axis for residual demand
    where='post',
    label="Residual Demand Curve"
)

# 2. Plot firm i's Supply Bid Function (step plot)
plt.step(
    firm_i_df["FIRM_CUMULATIVE_VOLUME"],  # X-axis for supply bid
    firm_i_df["PRICE"],                   # Y-axis for supply bid
    where='post',
    label=f"Supply Bid Function of {firm_i}"
)

# 3. Plot the MC Supply Curve (step plot) with data points
plt.step(
    dispatch_df_time_date_company['CumulativeCapacity'], 
    dispatch_df_time_date_company['AU$/MWh'], 
    where='post',
    label="MC Supply Curve"
)
plt.plot(
    dispatch_df_time_date_company['CumulativeCapacity'], 
    dispatch_df_time_date_company['AU$/MWh'],
    linestyle="None",   # No connecting lines, only markers
    marker="x",
    color="red",
    label="MC Data Points"
)

# Label the axes and title
plt.xlabel("Quantity (MW)")
plt.ylabel("Price ($/MWh)")  # Adjust the units if necessary
plt.title(f"Combined Market Curves for {firm_i} on {date}")

# Add legend and grid
plt.legend()
plt.grid(True)

plt.show()

## A test is to intersect the inelastic demand and the overall market supply function
I believe the above graph is currently wrong

In [72]:
firm_aggregated_bids

In [73]:
# Select only the needed columns
df = firm_aggregated_bids[["SETTLEMENTDATE", "PRICE", "VOLUME"]].copy()

# Sort by SETTLEMENTDATE and PRICE (ascending order)
df = df.sort_values(["SETTLEMENTDATE", "PRICE"], ascending=[True, True])

df = df[df["VOLUME"] != 0]

# Recalculate cumulative volume for each date
df["CUMULATIVE_VOLUME"] = df.groupby("SETTLEMENTDATE")["VOLUME"].cumsum()

pd.set_option("display.max_rows", None)
display(df)  # or simply: df

In [74]:
# Find the first row where the cumulative volume meets or exceeds the inelastic demand
try:
    mc_row = df[df["CUMULATIVE_VOLUME"] >= inelastic_demand].iloc[0]
    market_clearing_price = mc_row["PRICE"]
    print("Market Clearing Price:", market_clearing_price)
except IndexError:
    print("Inelastic demand exceeds the total cumulative volume. Check your data.")

In [75]:
result_row = df[df["PRICE"] == -34.99]
print(result_row)

In [76]:
# # 2. Create the Plot:
# plt.figure(figsize=(10, 6))

# # Plot the aggregated supply curve as a step function:
# plt.step(
#     firm_aggregated_bids["TOTAL_CUMULATIVE_VOLUME"],  # X-axis: cumulative volume
#     firm_aggregated_bids["PRICE"],                      # Y-axis: bid price
#     where='post',
#     label="Supply Curve"
# )

# # Plot the inelastic demand line (vertical line):
# plt.axvline(
#     x=inelastic_demand, 
#     color="red", 
#     linestyle="--", 
#     label="Inelastic Demand"
# )

# # Mark the intersection (market clearing point) with a marker:
# plt.scatter(
#     [inelastic_demand], 
#     [market_clearing_price], 
#     color="green", 
#     zorder=5, 
#     label=f"Market Clearing Price: {market_clearing_price:.2f}"
# )

# # Label the axes and add a title:
# plt.xlabel("Cumulative Volume (MW)")
# plt.ylabel("Price ($/MWh)")
# plt.title("Market Clearing: Intersection of Supply Curve and Inelastic Demand")

# # Add a legend and grid for clarity:
# plt.legend()
# plt.grid(True)
# plt.show()

## What do I notice
Need to truncate the residual demand to what is the capacity of firm i

## Price Data

In [77]:
# Get the price bids.
price_data = dynamic_data_compiler(start_time='2021/03/01 00:00:00',
                                   end_time='2021/04/10 00:00:00',
                                   table_name='DISPATCHPRICE',
                                   raw_data_location=raw_data_cache)

In [78]:
price_data.head()

In [79]:
date

In [80]:
test = price_data[price_data['SETTLEMENTDATE'] == '2021-04-07 18:05:00']
test 

## Thoughts

Hypothesis: Pricing and dispatch is done on a regional basis

In [81]:
from nem_bidding_dashboard import fetch_data
from nem_bidding_dashboard import fetch_and_preprocess

In [82]:
volume_bids = fetch_data.volume_bids(
    start_time='2022/01/01 00:00:00',
    end_time='2022/01/01 00:05:00',
    raw_data_cache=raw_data_cache)

In [83]:
bids = fetch_and_preprocess.bid_data(
    start_time='2021/04/07 18:00:00',
    end_time='2021/04/07 18:05:00',
    raw_data_cache=raw_data_cache)

In [84]:
bids

In [ ]:
# Inelastic demand already defined

bids_sorted = bids.sort_values(by="BIDPRICE", ascending=True) 

# Calculate cumulative sum of BIDVOLUMEADJUSTED for each settlement interval
bids_sorted['cum_bidvolumeadjusted'] = bids_sorted.groupby('INTERVAL_DATETIME')['BIDVOLUMEADJUSTED'].cumsum()

# Create the demand_function column as the inelastic demand minus the cumulative bid volume adjusted
bids_sorted['demand_function'] = inelastic_demand - bids_sorted['cum_bidvolumeadjusted']

bids_sorted

In [ ]:
# Find all bids for the 6:00-6:05pm interval
# INTERVAL_DATETIME in the bids data indicates the start of the 5-minute interval that the bids apply to
# We need to take the total demand and minus all the other rival bids. 
# If this is happening at a firm level, then that needs to be automated before moving on 
# so we need to find all the firm's supply bids or just do 

all_bids_for_single_day_auction = bids_with_duid[bids_with_duid['INTERVAL_DATETIME']  == '2021-04-09 18:00:00'] 
all_bids_for_single_day_auction